In [ ]:
#グーグルドライブ上のデータフレーム読み込み
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/

import pandas as pd
df_all = pd.read_pickle(r"df2001_2023.pkl")

# 血統登録番号リストを作成

In [ ]:
print(df_all['血統登録番号'].unique())
print(len(df_all['血統登録番号'].unique()))

[20106706 19106194 17103309 ... 98106548 98107180 98105525]
108829


In [ ]:
for i in range(2,11):
    # '血統登録番号'カラムの各要素を文字列に変換し、その長さが1である行の数を数える
    count = df_all[df_all['血統登録番号'].astype(str).apply(len) == i].count()
    # 結果を "i文字: count" の形式で表示
    print(f"{i}文字: {count['血統登録番号']}件")

2文字: 0件
3文字: 0件
4文字: 0件
5文字: 0件
6文字: 54921件
7文字: 682683件
8文字: 620504件
9文字: 0件
10文字: 0件


In [ ]:
# 血統登録番号カラムを文字列型に変換
df_all['血統登録番号'] = df_all['血統登録番号'].astype(str)

In [ ]:
# 血統登録番号の文字列の長さごとの内容を確認
for i in range(6,9):
    invalid_blood_rows = df_all[df_all['血統登録番号'].str.len() == i]
    print(f"{i}文字の内容：{invalid_blood_rows['血統登録番号'].unique()}")
    print(f"{i}文字の要素の数＜一意＞：{len(invalid_blood_rows['血統登録番号'].unique())}")

6文字の内容：['100231' '101470' '100965' ... '104287' '104074' '103502']
6文字の要素の数＜一意＞：4237
7文字の内容：['9100502' '9103405' '9102606' ... '1101159' '1107050' '1105145']
7文字の要素の数＜一意＞：40925
8文字の内容：['20106706' '19106194' '17103309' ... '98106548' '98107180' '98105525']
8文字の要素の数＜一意＞：63667


## スクレイピング時の血統登録番号は先頭2+(0埋め)血統登録番号の、全10桁のため、不足を補充

In [ ]:
# 血統登録番号の文字数に応じた処理を関数で定義

# 6桁：文字列2000を前に連結
# 7桁：文字列200を前に連結
# 8桁：文字列20を前に連結

def process_blood(x):
    if len(x) == 6:
        return '2000' + x
    elif len(x) == 7:
        return '200' + x
    elif len(x) == 8:
        return '20' + x
    else:
        return x  # 他のケースでは元の値を返す

# '血統登録番号'カラムの値を加工
#  applyメソッドは引数へ与えた関数を各値に処理して返す
df_all['血統登録番号'] = df_all['血統登録番号'].astype(str).apply(process_blood)

In [ ]:
 #血 統登録番号の文字列の長さごとの内容を再確認
for i in range(6,11):
    invalid_blood_rows = df_all[df_all['血統登録番号'].str.len() == i]
    print(f"{i}文字の内容：{invalid_blood_rows['血統登録番号'].unique()}")
    print(f"{i}文字の要素の数＜一意＞：{len(invalid_blood_rows['血統登録番号'].unique())}")

6文字の内容：[]
6文字の要素の数＜一意＞：0
7文字の内容：[]
7文字の要素の数＜一意＞：0
8文字の内容：[]
8文字の要素の数＜一意＞：0
9文字の内容：[]
9文字の要素の数＜一意＞：0
10文字の内容：['2020106706' '2019106194' '2017103309' ... '2098106548' '2098107180'
 '2098105525']
10文字の要素の数＜一意＞：108829


In [ ]:
# ユニークな血統登録番号をリストに格納
blood_id = df_all['血統登録番号'].unique()

# スクレイピングによるデータ取得処理

In [ ]:
#htmlを解析しデータを抽出するためのライブラリ
from bs4 import BeautifulSoup
#httpリクエストを行うためのライブラリ
#webページの情報を取得する
import requests

### スクレイピングの事前準備

In [ ]:
# 待機時間用ライブラリ
import time

In [ ]:
%%time

# スクレイピング結果を格納するリスト
Aptitude_data_list = []

# 探したい`<th>`タグのテキストのリストを作成
search_texts = ['コース適性', '距離適性', '脚質', '成長', '重馬場']

# 血統番号で馬番号をループする
for id in blood_id[0:len(blood_id)+1]:

    # URLの設定
    url = "https://db.netkeiba.com/horse/" + id + "/"

    # HTTPリクエストを行うための準備
    response = requests.get(url)

    # エンコーディング設定
    response.encoding = 'euc-jp'

    # HTMLの作成
    html = response.text

    # BeautifulSoupパーサーの作成
    soup = BeautifulSoup(html, "html.parser")

    # スクレイピング結果を格納するリスト
    results = []

    # HTML属性の解析および値の抽出
    # `<dl class="tekisei">`を見つける
    tekisei_dl = soup.find('dl', class_='tekisei')

    if tekisei_dl:

        # `<dl>`内の`<tr>`タグをループ処理
        for tr in tekisei_dl.find_all('tr'):

            th = tr.find('th')

            if th and th.text in search_texts:

                # `<tr>`タグ内の`<td>`タグから`<img>`タグを探す
                width_attrs = [img['width'] for img in tr.find_all('img', width=True)]

                # 二番目の`width`属性の値を取得して保存
                if len(width_attrs) >= 2:
                    results.append(width_attrs[1])

    Aptitude_data_list.append({id: results})

    # 待ち時間を作成
    time.sleep(0.5)

# 結果をpickle形式で保存
with open("data2001_5001.pickle", "wb") as f:
    pickle.dump(Aptitude_data_list, f)